In [5]:
import requests
menu_url = 'https://tienda.consum.es/api/rest/V1.0/shopping/category/menu'
productos_url = 'https://tienda.consum.es/api/rest/V1.0/catalog/product'
respuesta = requests.get(menu_url)
print(respuesta)
data = respuesta.json()
#print(data)
dic = {}


<Response [200]>


In [9]:
for elem in data:
    categorias = elem['subcategories']
    for category in categorias:
        category_id = category['id']
        category_name = category['name']
        if category_name not in dic:
            dic[category_name] = {}

        page = 1
        limit = 100
        while True:
            params = {
                'categories': category_id,
                'page': page,
                'limit': limit,
                'offset':(page-1)*limit, #a partir de donde tiene que empezar la pagina
                'showProducts': 'true'
            }
        
            r = requests.get(productos_url, params=params).json()
            products = r['products']

            if not products:
                break  # no hay más páginas

            for p in products:
                producto_id = p['id']
                nombre = p['productData']['name']
                precio = p['priceData']['prices'][0]['value']['centUnitAmount']

                dic[category_name][producto_id] = {
                    'nombre': nombre,
                    'precio': precio
                }

            page += 1
print(dic)
        
    


{'Aperitivos y frutos secos': {4668: {'nombre': 'Nuez Cáscara Granel', 'precio': 6.2}, 4867: {'nombre': 'Aperitivo de Maíz Pelotazos Sabor Queso', 'precio': 14.54}, 5055: {'nombre': 'Aperitivo Frito Pandilla Sabor Queso', 'precio': 25.2}, 31828: {'nombre': 'Palomitas de Chocolate Bolsa', 'precio': 14.0}, 5210: {'nombre': 'Aceitunas Manzanilla sin Hueso  Bolsa', 'precio': 12.67}, 5443: {'nombre': 'Apetinas Kétchup Bolsa', 'precio': 14.33}, 5447: {'nombre': 'Patatas Fritas Lisas Artesanas Bolsa Papel', 'precio': 11.07}, 5528: {'nombre': 'Aceitunas Negras Hueso Lata', 'precio': 6.76}, 5529: {'nombre': 'Aceitunas Negras sin Hueso  Lata', 'precio': 7.93}, 5530: {'nombre': 'Aceitunas Rellenas de Anchoas', 'precio': 7.67}, 5532: {'nombre': 'Aceitunas Rellenas de Anchoas Pack de 3', 'precio': 12.67}, 5535: {'nombre': 'Pipas Tostadas con Sal Bolsa', 'precio': 5.8}, 5708: {'nombre': 'Aceitunas Rellenas de Anchoas Pack de 3', 'precio': 22.33}, 5709: {'nombre': 'Aceitunas Rellenas Anchoa Cantábric

In [11]:
import csv

with open("consum_precios.csv", mode="w", newline="", encoding="utf-8") as file:
    writer = csv.writer(file)
    writer.writerow(["categoria", "ID producto", "producto", "precio"])

    for categoria, productos in dic.items():
        for producto_id, info in productos.items():
            writer.writerow([
                categoria,
                producto_id,
                info['nombre'],
                info['precio']
            ])

